In [1]:
import pandas
from IPython.display import display, HTML
import random

In [2]:
platform_da = ['/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/Raw/export_dataset_20200307-064859.csv',
'/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/Raw/export_resource_20200307-065602.csv',
'/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/Raw/export_reuse_20200307-065759.csv',
'/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/Raw/export_entity.csv',
'/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/Raw/export_user_global.csv']

list_var = list()
for i,e in enumerate(platform_da):
    try:
        list_var.append((e.split('/')[-1].split('_')[1],pandas.read_csv(e, sep = ';').keys().values.tolist()))
    except:
        list_var.append((e.split('/')[-1].split('_')[1],pandas.read_csv(e, sep = ',').keys().values.tolist()))


list_files = list()
for i,e in enumerate(platform_da):
    print(e)
    try:
        df = pandas.read_csv(e, sep = ';')
    except:
        df = pandas.read_csv(e, sep = ',')
    print(df.keys())
    df['file'] = e.split('/')[-1].split('_')[1]
    list_files.append((e.split('/')[-1].split('_')[1],df))
    if 'ID' in df.keys():
        df['id'] = df['ID']

/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/Raw/export_dataset_20200307-064859.csv
Index(['id', 'title', 'slug', 'url', 'organization', 'organization_id',
       'description', 'frequency', 'license', 'temporal_coverage.start',
       'temporal_coverage.end', 'spatial.granularity', 'spatial.zones',
       'private', 'featured', 'created_at', 'last_modified', 'tags',
       'metric.discussions', 'metric.followers', 'metric.reuses',
       'metric.issues', 'metric.views'],
      dtype='object')
/Volumes/GoogleDrive/My Drive/Open Data Platform/DATA/Raw/export_resource_20200307-065602.csv
Index(['dataset.id', 'dataset.title', 'dataset.slug', 'dataset.url',
       'dataset.organization', 'dataset.organization_id', 'dataset.license',
       'dataset.private', 'id', 'url', 'title', 'description', 'filetype',
       'format', 'mime', 'filesize', 'checksum.type', 'checksum.value',
       'created_at', 'modified', 'downloads'],
      dtype='object')
/Volumes/GoogleDrive/My Drive/Open D

In [55]:
import collections
#print(collections.Counter([a.split(' ')[0] for a in list_files[-1][1]['Date de création de compte'].sort_values().values.tolist() if a[:4] == '2020']))

pandas.DataFrame(collections.Counter(['-'.join(a.split(' ')[0].split('-')[:2]) for a in list_files[-1][1]['Date de création de compte'].sort_values().values.tolist() if a[:4] == '2020']).most_common(12)
, columns = ['date', 'count']).sort_values('date').sum()


date     2020-012020-022020-032020-042020-052020-062020-07
count                                                 5672
dtype: object

In [43]:
collections.Counter([a[1] for a in list_files[-1][1][['Date de création de compte','Abonnement à la newsletter']].values.tolist() if a[0][:4] == '2020'])

Counter({'Non': 4535, 'Oui': 1137})

In [59]:
collections.Counter([a[1] for a in list_files[-1][1][['Date de création de compte','Accepte de transmettre ses coordonnées']].values.tolist() if a[0][:4] == '2020'])

Counter({'Non': 5672})

In [62]:
list_files[-1][1][['ID',"Profil d'entrepreneur"]].groupby("Profil d'entrepreneur").count()

,ID
Profil d'entrepreneur,
association,68
big,23
cooperative,9
ong,55
small,474
startup,975


In [67]:
list_files[-1][1]['Type'].values.tolist()

[nan,
 'investisseur',
 'entrepreneur, ',
 'entrepreneur, Ingenieur Génie Civil ',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'entrepreneur, ',
 'investisseur, etudiant, freelance, ',
 'entrepreneur, consultant, freelance, ',
 nan,
 nan,
 'entrepreneur, Production locale et transformation ',
 'entrepreneur',
 'entrepreneur, ',
 nan,
 'salarie, benevole, recherche_emploi, freelance, Assistante de Dir, Commerciale ou Jurid et coiffeuse évenementiels à mon cpte pdt 3 ans => 2017',
 nan,
 'etudiant, recherche_emploi, ',
 'entrepreneur, ',
 nan,
 nan,
 nan,
 'entrepreneur, ',
 nan,
 'salarie, ',
 'entrepreneur',
 nan,
 'entrepreneur, ',
 nan,
 'entrepreneur, ',
 'entrepreneur, ',
 nan,
 nan,
 nan,
 nan,
 'Expert-comptable mémorialiste',
 nan,
 'Entrepreneur',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'entrepreneur, investisseur, journaliste, ',
 nan,
 'entrepreneur, consultant, ',
 'entrepreneur, ',
 nan,
 nan,
 nan,
 'entrepreneur, ',
 'entrepreneur, ',
 nan,
 'entrepreneur, ',
 'entrepreneur, 

In [ ]:
### var = []
for element in list_var:
    var = var + [[element[0], e] for e in element[1]]

var_map = pandas.DataFrame(var, columns =['type_file', 'variable'])
var_map['index'] = var_map['type_file']+'_'+var_map['variable']
var_map = var_map.set_index('index')

var_map

In [4]:
def completion_rate(type_file):
    target = [a for a in filter(lambda x: x[0]==type_file, list_files)][0][1]
    rate = 100-target.isna().sum()*100/len(target)#
    rate = pandas.DataFrame(rate, columns =['completion rate'])
    rate['file'] = list(set(target['file']))[0]
    rate = rate.reset_index()
    rate['variable'] = rate['index']
    rate['index'] = rate['file']+'_'+rate['variable']
    rate = rate[['completion rate', 'file', 'index']].set_index('index')
    return rate

def type_variable(type_file):
    target = [a for a in filter(lambda x: x[0]==type_file, list_files)][0][1]
    header = [['index', 'file', 'variable', 'type', 'nombre de type var']]
    for e in target:
        t_var = list({str(type(a)) for a in set(target[e])})
        nbr_t = len(t_var)
        t_var = '/'.join(t_var)
        header = header +[[type_file +'_'+e, type_file, e,t_var,nbr_t]]
    target = pandas.DataFrame(header[1:], columns = header[0]).set_index('index')
    target = target.drop(['variable', 'file'], axis =1)
    return target

def describe_data(type_file):
    target = [a for a in filter(lambda x: x[0]==type_file, list_files)][0][1]
    target = target.describe().transpose().reset_index()
    target['variable'] = target['index']
    target['index'] = type_file + '_' + target['index']
    target = target.set_index('index')
    target = target.drop('variable', axis =1 )
    return target


In [5]:
df_target = var_map.join(pandas.concat([completion_rate(e) for e in set(var_map['type_file'])], axis = 0))
df_target = df_target.join(pandas.concat([describe_data(e) for e in set(var_map['type_file'])], axis = 0))
df_target = df_target.join(pandas.concat([type_variable(type_file)for type_file in set(var_map['type_file'])], axis = 0))
df_target['platform'] = 'platform DA'


In [6]:
df_target.fillna('', inplace = True)

In [7]:
from DA_IO import Gspreadsheet
worksheet_name = 'audit data'
gsp = Gspreadsheet(worksheet_name)
#gsp.create()
sheet_name = 'plateforme DA'
gsp.add_sheet(sheet_name)
gsp.df_to_sheet(df = df_target, sheet_name = sheet_name)

https://docs.google.com/spreadsheets/d/1f4nrE397_sFVEPAuUYnUAdwmBs_HTMh56dnthT3w84A/edit#gid=0
